In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error
import pandas as pd

In [ ]:


df = pd.read_csv('dynamic_pricing.csv')



# Encode categorical variables
label_encoders = {}
for column in ['Location_Category', 'Customer_Loyalty_Status', 'Time_of_Booking', 'Vehicle_Type']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Split the data into features and target
X = df.drop('Historical_Cost_of_Ride', axis=1)
y = df['Historical_Cost_of_Ride']

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and validation sets
def get_data_splits(X, y, valid_fraction=0.2):
    valid_size = int(len(X) * valid_fraction)
    X_train, X_valid = X[:-valid_size], X[-valid_size:]
    y_train, y_valid = y[:-valid_size], y[-valid_size:]
    return X_train, X_valid, y_train, y_valid

X_train, X_valid, y_train, y_valid = get_data_splits(X_scaled, y)

# Define the neural network
def neural_network(nodes, input_length):
    model = Sequential()
    model.add(Dense(nodes, input_dim=input_length, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='relu'))
    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=["mae"])
    return model

# Train the neural network
model = neural_network(23, X_train.shape[1])
history = model.fit(X_train, y_train, epochs=100, batch_size=256, validation_data=(X_valid, y_valid), verbose=0)

# Plot training and validation loss
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
plt.plot(loss_values, label='training loss')
plt.plot(val_loss_values, label='val loss')
plt.legend()
plt.show()

# Evaluate the model
y_train_pred = model.predict(X_train)
y_valid_pred = model.predict(X_valid)
train_mae = mean_absolute_error(y_train, y_train_pred)
valid_mae = mean_absolute_error(y_valid, y_valid_pred)
print("The R2 score on the Train set is: ", r2_score(y_train, y_train_pred))
print("The R2 score on the Valid set is: ", r2_score(y_valid, y_valid_pred))
print("The mae on the Train set is:\t{:0.3f}".format(train_mae))
print("The mean of the Train set is: ", y_train.mean())
print("The percentage of mae on Train set is: ", (train_mae / y_train.mean()) * 100)
print("The mae on the Valid set is:\t{:0.3f}".format(valid_mae))
print("The mean of the Valid set is: ", y_valid.mean())
print("The percentage of mae on Valid set is: ", (valid_mae / y_valid.mean()) * 100)

# Save the model
model.save("final_model.h5")
